In [4]:
import sys
import json
sys.path.append('../')
from data_processing import preprocess

In [51]:
data = preprocess.KaggleDataset().process_data()

In [54]:
len(data)

200

In [9]:
for i in data:
    print(i['posts'])

[' ', "I dreamed a lot of weird stuff but generally my dream always sent me back to school for some reason to meet someone I haven't met in a long while. Previously I dreamed of flying in a machine...", 'Maybe some headless dolls?', 'People like thank yous as much as the thumbs up icon so keep sending them.', 'Possible', 'Try this.  717794', 'No more spam? Nuuuuuuuuuu', 'For some people, life is a buffet. For others, their world is a set meal.   LOL.', 'I currently desire pancakes.', 'Fi dom signature.', 'Neither heartlessness and indecision for not following your guts will get you anywhere.', 'Hahahahahaha!!!', 'What makes you so certain that only s and not s can be writers? Both are good with written words. Just a genuine question. :happy:', '', '', 'Seems likely.', '  ', '- If running the treadmill is not for you, you could either try the rowing machine or if there is a public pool available, try swimming. They are relatively low-impact exercises but would still give...', "I was so 

In [2]:
from sentence_transformers.util import cos_sim
from transformers import AutoTokenizer
def get_json_list(file_path, start_at=0, end_at=None):
    with open(file_path, "r") as f:
        json_list = []
        for idx, line in enumerate(f):
            if end_at is not None and idx >= end_at:
                return json_list
            elif idx < start_at:
                continue
            json_list.append(json.loads(line))
        return json_list
    
def get_similar_embeddings_list(embeddings):
    similarity_pairs = []
    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            similarity = cos_sim(embeddings[i], embeddings[j]).item()
            similarity_pairs.append(((i, j), similarity))

    # similarity_scores.sort(key=lambda x: x[1], reverse=True)

    return similarity_pairs

def select_least_pair(data, max_tokens=3000):
    tokenizer = AutoTokenizer.from_pretrained("fxmarty/tiny-llama-fast-tokenizer")
    similarity_pairs = get_similar_embeddings_list(data['embeddings'])
    sorted_pairs = sorted(similarity_pairs, key=lambda x: x[1], reverse=True) 
    data_length = get_total_length(data)
    selected_posts = []
    total_length = 0
    for (idx1, idx2), similarity in sorted_pairs:
        if total_length >= max_tokens:
            break

        post1 = data['posts'][idx1]
        post2 = data['posts'][idx2]
        length_post1 = len(tokenizer.tokenize(post1))
        length_post2 = len(tokenizer.tokenize(post2))

        if post1 not in [post['text'] for post in selected_posts] and total_length + length_post1 <= max_tokens:
            selected_posts.append({"index": idx1, "text": post1, "similarity": similarity})
            total_length += length_post1

        if post2 not in [post['text'] for post in selected_posts] and total_length + length_post2 <= max_tokens:
            selected_posts.append({"index": idx2, "text": post2, "similarity": similarity})
            total_length += length_post2

    return selected_posts, data_length

def get_total_length(data):
    tokenizer = AutoTokenizer.from_pretrained("fxmarty/tiny-llama-fast-tokenizer")
    total_length = 0
    for post in data['posts']:
        total_length += len(tokenizer.tokenize(post))
    return total_length

In [120]:
data = get_json_list('text_embeddings.json')

In [127]:
len(data[0][0]['embeddings'])

53

In [129]:
pairs, data_length = select_least_pair(data[0][0])
data_length

788

In [137]:
all_data_length = []
all_pairs = []
for i in data[0]:
    pairs, length = select_least_pair(i, max_tokens=3000)
    all_pairs.append(pairs)
    all_data_length.append(length)
all_data_length

[788,
 2284,
 1492,
 1969,
 2199,
 1775,
 2176,
 2285,
 1980,
 2014,
 2331,
 1729,
 2167,
 1830,
 2365,
 1603,
 1921,
 1952,
 1623,
 2299,
 2064,
 1832,
 2241,
 752,
 2274,
 1630,
 2183,
 2279,
 2050,
 1853,
 1850,
 2207,
 1829,
 1663,
 1659,
 2333,
 2075,
 1817,
 1604,
 2003,
 2060,
 816,
 2067,
 1810,
 1710,
 1441,
 2147,
 1435,
 1743,
 1540,
 2289,
 1673,
 1212,
 843,
 2054,
 2273,
 2003,
 2132,
 2121,
 2357,
 1169,
 2426,
 377,
 1629,
 1996,
 1539,
 2134,
 897,
 991,
 2219,
 1235,
 1532,
 1751,
 1493,
 2439,
 320,
 2467,
 2308,
 2180,
 1751,
 1912,
 1857,
 2431,
 2140,
 1794,
 2291,
 2417,
 1443,
 1404,
 1735,
 2031,
 1639,
 1018,
 2258,
 990,
 2137,
 2106,
 2186,
 2047,
 1943,
 2036,
 1893,
 1905,
 962,
 1631,
 2086,
 1817,
 2202,
 1471,
 1718,
 2066,
 2006,
 981,
 2033,
 2042,
 1818,
 1530,
 1729,
 1585,
 2067,
 1830,
 2228,
 1526,
 2630,
 2355,
 1608,
 1738,
 746,
 2196,
 1563,
 1033,
 2227,
 1821,
 1581,
 2070,
 1754,
 1899,
 2111,
 2864,
 2008,
 2316,
 903,
 2163,
 1401,
 2237

In [138]:
max(all_data_length)

2864

In [148]:
all_pairs[0]

[{'index': 21,
  'text': 'Your logic will be disabled soon.. :laughing:',
  'similarity': 0.6672337651252747},
 {'index': 22, 'text': ':laughing: Yes.', 'similarity': 0.6672337651252747},
 {'index': 9,
  'text': 'Neither heartlessness and indecision for not following your guts will get you anywhere.',
  'similarity': 0.6174330711364746},
 {'index': 25,
  'text': "It takes more guts to be empathetic when you see people that annoys you everyday. Being heartless is so easy. It's very easy to not care.",
  'similarity': 0.6174330711364746},
 {'index': 3, 'text': 'Possible', 'similarity': 0.6089472770690918},
 {'index': 12, 'text': 'Seems likely.', 'similarity': 0.6089472770690918},
 {'index': 7,
  'text': 'I currently desire pancakes.',
  'similarity': 0.45148834586143494},
 {'index': 17,
  'text': "I'm hungry just looking at them..",
  'similarity': 0.45148834586143494},
 {'index': 30,
  'text': "Sometimes when it becomes work,it would lost its' enjoyment especially when faced with an emp

In [50]:
data[0][0]['post'][14]

''

In [89]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("fxmarty/tiny-llama-fast-tokenizer")

tokenizer_config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [91]:
tokenizer.tokenize(data[0][0]['posts'][2])

['▁People',
 '▁like',
 '▁thank',
 '▁you',
 's',
 '▁as',
 '▁much',
 '▁as',
 '▁the',
 '▁thumb',
 's',
 '▁up',
 '▁icon',
 '▁so',
 '▁keep',
 '▁sending',
 '▁them',
 '.']

#### Explore selected_output.json


In [11]:
sd = get_json_list('/home/wenkail/ANLP_Final/src/data/selected_output_max_length_1000.json')

In [12]:
sd[0][0]

{'pairs': [{'index': 21,
   'text': 'Your logic will be disabled soon.. :laughing:',
   'similarity': 0.6672337651252747},
  {'index': 22, 'text': ':laughing: Yes.', 'similarity': 0.6672337651252747},
  {'index': 9,
   'text': 'Neither heartlessness and indecision for not following your guts will get you anywhere.',
   'similarity': 0.6174329519271851},
  {'index': 25,
   'text': "It takes more guts to be empathetic when you see people that annoys you everyday. Being heartless is so easy. It's very easy to not care.",
   'similarity': 0.6174329519271851},
  {'index': 3, 'text': 'Possible', 'similarity': 0.6089472770690918},
  {'index': 12, 'text': 'Seems likely.', 'similarity': 0.6089472770690918},
  {'index': 7,
   'text': 'I currently desire pancakes.',
   'similarity': 0.45148834586143494},
  {'index': 17,
   'text': "I'm hungry just looking at them..",
   'similarity': 0.45148834586143494},
  {'index': 30,
   'text': "Sometimes when it becomes work,it would lost its' enjoyment espe

In [14]:
sd_length = []
for i in sd[0]:
    sd_length.append(i['new_length'])


In [16]:
sd_length

[788,
 1000,
 993,
 999,
 999,
 998,
 994,
 989,
 994,
 992,
 991,
 1000,
 998,
 1000,
 996,
 997,
 999,
 998,
 995,
 1000,
 999,
 1000,
 998,
 752,
 1000,
 998,
 999,
 997,
 999,
 1000,
 995,
 1000,
 1000,
 1000,
 1000,
 998,
 999,
 999,
 1000,
 998,
 995,
 816,
 1000,
 999,
 1000,
 1000,
 998,
 998,
 997,
 998,
 995,
 994,
 1000,
 843,
 999,
 999,
 995,
 1000,
 1000,
 1000,
 998,
 982,
 377,
 1000,
 1000,
 1000,
 1000,
 897,
 991,
 997,
 999,
 999,
 999,
 1000,
 996,
 320,
 997,
 995,
 998,
 995,
 999,
 999,
 1000,
 999,
 1000,
 997,
 999,
 997,
 1000,
 1000,
 996,
 1000,
 966,
 995,
 990,
 999,
 1000,
 995,
 1000,
 996,
 998,
 994,
 998,
 962,
 999,
 986,
 999,
 997,
 997,
 997,
 999,
 1000,
 981,
 997,
 998,
 997,
 999,
 1000,
 996,
 1000,
 1000,
 993,
 1000,
 1000,
 997,
 999,
 999,
 746,
 999,
 999,
 994,
 997,
 1000,
 995,
 999,
 1000,
 999,
 995,
 990,
 1000,
 995,
 888,
 998,
 1000,
 987,
 1000,
 997,
 1000,
 1000,
 993,
 992,
 998,
 999,
 999,
 1000,
 999,
 994,
 61,
 998,
 9